In [18]:
from animalai.envs.arena_config import Vector3, RGB, Item, Arena, ArenaConfig
from animalai.envs.environment import AnimalAIEnvironment
from mlagents_envs.exception import UnityCommunicationException

from typing import List
from animalai.communicator_objects import (
    ArenasConfigurationsProto,
    ArenaConfigurationProto,
    ItemToSpawnProto,
    VectorProto,
)

from typing import List
import yaml
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

import networkx as nx
from networkx import grid_graph

# Notebook to go from a matrix to an AnimalAI environment
In this notebook we just play with how the AnimalAI environments are created, and we design functions to go from a AnimalAI environment to a numpy matrix and viceversa.

In [19]:
## We redefine the class ArenaConfig to MyArenaConfig so instead of reading a .yaml file.
## it reads the arena object we create

class MyArenaConfig(yaml.YAMLObject):
    yaml_tag = u"!ArenaConfig"

    def __init__(self, my_arena: Arena = None):

        self.arenas = {-1: my_arena}

    def to_proto(self, seed: int = -1) -> ArenasConfigurationsProto:
        arenas_configurations_proto = ArenasConfigurationsProto()
        arenas_configurations_proto.seed = seed

        for k in self.arenas:
            arenas_configurations_proto.arenas[k].CopyFrom(self.arenas[k].to_proto())

        return arenas_configurations_proto

In [ ]:
## For simplicity, we only modify the position of the goal (position of agent is fixed)

def create_arena(x, y):
    '''
    Create an arena with only one agent and one goal.
    :param x: float. x position goal
    :param x: float. y position goal
    '''
    
    # Create agent
    position_agent = Vector3(x = 20, y = 0, z = 20)
    rotation_agent = 0
    agent = Item(name = 'Agent', positions = [position_agent], rotations = [rotation_agent])

    # Create the goal
    position_goal = Vector3(x = x, y = 0, z = y)
    sizes_goal = Vector3(x = 1, y = 1, z = 1)
    goal = Item(name = 'GoodGoal', positions = [position_goal], sizes = [sizes_goal])

    # Define list of items
    items = [agent, goal]

    # Create Arena
    my_arena = Arena(t=250, items=items, pass_mark = 0, blackouts = None)

    # create arena configuration
    my_config = MyArenaConfig(my_arena)
    
    return my_config

## 1. Generate matrix from arena

In [ ]:
my_arena = create_arena(x=39, y=2)

In [ ]:
def matrix_from_arena(my_arena):

    for k in my_arena.arenas:
        item_types = []
        item_types_value = []
        arena_m = np.zeros((40, 40)) - 1
        for (i, item) in enumerate(my_arena.arenas[k].items):
            # save each type of item
            item_types.append(item.name)
            item_types_value.append(i)
            # positions (in case one item takes more than one position)
            for (p, pos) in enumerate(item.positions):
                x = pos.x
                y = pos.z
                if item.sizes:
                    # if we have the size of the item
                    x_size = item.sizes[p].x
                    y_size = item.sizes[p].z

                    # where the item starts and ends
                    if x_size > 1:
                        start_x = x - int(x_size/2)
                        end_x = x + int(x_size/2)
                    else:
                        start_x = x
                        end_x = x
                    if y_size > 1:
                        start_y = y - int(y_size/2)
                        end_y = y + int(y_size/2)
                    else:
                        start_y = y
                        end_y = y

                    # paint figure
                    for x_ in range(start_x, end_x+1):
                        for y_ in range(start_y, end_y+1):
                            arena_m[x_, y_] = i
                else:
                    arena_m[x, y] = i

        fig = plt.imshow(arena_m, cmap=plt.get_cmap('Accent'))

        items = np.concatenate((['Arena'], item_types))

        # unique values
        values = np.unique(arena_m.ravel())
        # colormap used by imshow
        colors = [fig.cmap(fig.norm(value)) for value in values]
        # create a patch (proxy artist) for every color 
        patches = [mpatches.Patch(color=colors[i], label="{l}".format(l=items[i]) ) for i in range(len(values)) ]
        # put those patched as legend-handles into the legend
        plt.legend(handles=patches, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0. )

        plt.grid(False)
        plt.show()
        
        return arena_m

In [ ]:
my_arena = ArenaConfig('../configurations/arena_configurations/cylinder_task.yml')
arena_m = matrix_from_arena(my_arena)

In [ ]:
my_arena = ArenaConfig('../configurations/curriculum/2.yml')
matrix_from_arena(my_arena)

## 2. Add item to an existing arena

In [ ]:
def update_arena(old_arena, new_item):
    # we add the new item
    old_arena.items.append(new_item)

    # save the model in a arena and create new config
    new_arena = Arena(t=old_arena.t, items=old_arena.items, pass_mark = old_arena.pass_mark, blackouts = None)
    
    return new_arena

In [ ]:
def create_wall(position = [2, 0, 20], size = [1, 1, 1]):
    '''
    Given the position and the size of the wall, it creates the item
    '''
    
    # create new item: wall
    position_wall = Vector3(x = position[0], y = position[1], z = position[2])
    sizes_wall = Vector3(x = size[0], y = size[1], z = size[2])
    
    return Item(name = 'Wall', positions = [position_wall], sizes = [sizes_wall])

In [ ]:
# Create Arena
my_arena = ArenaConfig('../configurations/curriculum/2.yml')

print('Old Arena')
matrix_from_arena(my_arena)

# save old arena
old_arena = my_arena.arenas[-1]

# create wall
wall = create_wall(position = [2, 0, 10], size = [5, 1, 10])

# update
new_arena = update_arena(old_arena, new_item=wall)
new_arena = MyArenaConfig(new_arena)

print('New Arena')
matrix_from_arena(new_arena)

## 3. Go from matrix to AnimalAI arena

In [20]:
# generate example of matrix with 1 agent (value:1), 1 goal (value:2) and 1 wall (value:3)
# background arena value is 0
matrix = np.zeros((40,40))
matrix[1, 15] = 1
matrix[1, 25] = 2
matrix[25, 35] = 3
matrix[10, 12] = 3
matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [31]:
# get agent position
position = np.where(matrix==1)
position_goal = Vector3(x = position[0][0], y = 0, z = position[1][0])
sizes_goal = Vector3(x = 2, y = 2, z = 2)
goal = Item(name = 'GoodGoal', positions = [position_goal], sizes = [sizes_goal])

position = np.where(matrix==2)
position_agent = Vector3(x = position[0][0], y = 0, z = position[1][0])
agent = Item(name = 'Agent', positions = [position_agent], rotations = [0])


position = np.where(matrix==3)
position_wall= Vector3(x = position[0][0], y = 0, z = position[1][0])
sizes_wall = Vector3(x = 2, y = 2, z = 2)
color_wall = RGB(r = 200, g = 0, b = 4)
wall = Item(name = 'Wall', positions = [position_wall], sizes = [sizes_wall], colors = [color_wall])

# Define list of items
items = [agent, goal, wall]

# Create Arena
my_arena = Arena(t=250, items=items, pass_mark = 0, blackouts = None)
my_config = MyArenaConfig(my_arena)

In [ ]:
len(np.where(matrix == 3))

In [ ]:
def matrix2arena(matrix):
    '''Given an input numpy matrix of the arena image, it create an AnimalAI environment configuration.
    Matrix values --> 0: arena, 1: goal, 2: agent, 3: wall'''
    if matrix.shape != (40, 40):
        print('Input matrix should have shape 40x40')
        return 
    
    items = []
    
    position = np.where(matrix==1)
    position_goal = Vector3(x = position[0][0], y = 0, z = position[1][0])
    sizes_goal = Vector3(x = 2, y = 2, z = 2)
    goal = Item(name = 'GoodGoal', positions = [position_goal], sizes = [sizes_goal])
    items.append(goal)

    position = np.where(matrix==2)
    position_agent = Vector3(x = position[0][0], y = 0, z = position[1][0])
    agent = Item(name = 'Agent', positions = [position_agent])
    items.append(agent)

    position = np.where(matrix==3)
    for i in range(len(position)):
        position_wall= Vector3(x = position[0][i], y = 0, z = position[1][i])
        sizes_wall = Vector3(x = 2, y = 2, z = 2)
        wall = Item(name = 'Wall', positions = [position_wall], sizes = [sizes_wall])
        items.append(wall)

    # Create Arena
    my_arena = Arena(t=250, items=items, pass_mark = 0, blackouts = None)
    my_config = MyArenaConfig(my_arena)
        
    
    return my_config

In [ ]:
my_config = matrix2arena(matrix)

In [ ]:
my_config.to_proto()

In [1]:
from utils import *

In [3]:
matrix = np.zeros((40,40))
matrix[1, 15] = 1
matrix[1, 25] = 2
matrix[25, 35] = 3
matrix[10, 12] = 3
my_config = matrix2arena(matrix)
my_config.to_proto()

arenas {
  key: -1
  value {
    t: 250
    items {
      name: "GoodGoal"
      positions {
        x: 1.0
        z: 15.0
      }
      sizes {
        x: 2.0
        y: 2.0
        z: 2.0
      }
    }
    items {
      name: "Agent"
      positions {
        x: 1.0
        z: 25.0
      }
    }
    items {
      name: "Wall"
      positions {
        x: 10.0
        z: 12.0
      }
      sizes {
        x: 2.0
        y: 2.0
        z: 2.0
      }
    }
    items {
      name: "Wall"
      positions {
        x: 25.0
        z: 35.0
      }
      sizes {
        x: 2.0
        y: 2.0
        z: 2.0
      }
    }
  }
}
seed: -1

In [32]:
# visualize evironment just created

try:
    environment = AnimalAIEnvironment(
            file_name='../env/AnimalAI',
            base_port=5008,
            arenas_configurations=my_config,
            play=True,
            worker_id = 5,
        )
except UnityCommunicationException:
    # you'll end up here if you close the environment window directly
    # always try to close it from script
    environment.close()

In [33]:
if environment:
    environment.close() # takes a few seconds

In [16]:
m = np.zeros((38, 38)) + 1
m

array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]])

In [17]:
result = np.zeros((40, 40))
# actually you can also use result = np.zeros_like(b) 
# but that also copies the dtype not only the shape

result[1:m.shape[0]+1,1:m.shape[1]+1] = m
result

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 1., ..., 1., 1., 0.],
       [0., 1., 1., ..., 1., 1., 0.],
       ...,
       [0., 1., 1., ..., 1., 1., 0.],
       [0., 1., 1., ..., 1., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [14]:
result[1:3, 1:3]

array([[0., 0.],
       [0., 0.]])